<a href="https://colab.research.google.com/github/Amitgm/LLm/blob/main/Graph%20Database/Graph_Database_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --quiet langchain-community langchain langchain-openai neo4j

In [53]:
!pip install --upgrade  langchain-groq

In [54]:
NEO4J_URI="neo4j+s://d24cab19.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="PECThut0LIIl6dC0DzH19bykGYHaAAWPCLHIr3djEUE"
# AURA_INSTANCEID=d24cab19
# AURA_INSTANCENAME=Instance01

In [55]:
!pip install langchain_community

In [57]:
import os
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [56]:
from langchain_community.graphs import Neo4jGraph

In [58]:
graph = Neo4jGraph(
    url = os.environ["NEO4J_URI"],
    username = os.environ["NEO4J_USERNAME"],
    password = os.environ["NEO4J_PASSWORD"]

)

In [59]:
graph

In [60]:
from langchain_groq import ChatGroq

In [61]:
grok_api_key = "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"

In [62]:
llm = ChatGroq(model_name="Gemma2-9b-it")

In [63]:
from langchain_core.documents import Document

In [64]:
text = '''

Elon Reeve Musk was born on June 28, 1971, in Pretoria, South Africa's administrative capital.
He is of British and Pennsylvania Dutch ancestry. His mother, Maye (née Haldeman), is a model and dietitian born in Saskatchewan, Canada, and raised in South Africa.
His father, Errol Musk, is a South African electromechanical engineer, pilot, sailor, consultant, emerald dealer, and property developer, who partly owned a rental lodge at the Timbavati Private Nature Reserve.
Elon has a younger brother, Kimbal, and a younger sister, Tosca.
Elon has four paternal half-siblings.

'''

In [65]:
documents = [Document(page_content=text)]

In [66]:
!pip install --upgrade langchain-experimental

In [29]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

In [25]:
documents

[Document(page_content="\n\nElon Reeve Musk was born on June 28, 1971, in Pretoria, South Africa's administrative capital. \nHe is of British and Pennsylvania Dutch ancestry. His mother, Maye (née Haldeman), is a model and dietitian born in Saskatchewan, Canada, and raised in South Africa. \nHis father, Errol Musk, is a South African electromechanical engineer, pilot, sailor, consultant, emerald dealer, and property developer, who partly owned a rental lodge at the Timbavati Private Nature Reserve. \nElon has a younger brother, Kimbal, and a younger sister, Tosca.\nElon has four paternal half-siblings.\n\n")]

In [67]:
llmtransformer = LLMGraphTransformer(llm=llm)

In [68]:
graph_documents = llmtransformer.convert_to_graph_documents(documents)

In [69]:
graph_documents

[GraphDocument(nodes=[Node(id='Elon Reeve Musk', type='Person'), Node(id='June 28, 1971', type='Date'), Node(id='Pretoria', type='Location'), Node(id='South Africa', type='Location'), Node(id='British', type='Nationality'), Node(id='Pennsylvania Dutch', type='Ethnicity'), Node(id='Maye (Née Haldeman)', type='Person'), Node(id='Saskatchewan', type='Location'), Node(id='Canada', type='Location'), Node(id='Errol Musk', type='Person'), Node(id='Timbavati Private Nature Reserve', type='Location'), Node(id='Kimbal', type='Person'), Node(id='Tosca', type='Person')], relationships=[Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='June 28, 1971', type='Date'), type='BORN'), Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Pretoria', type='Location'), type='BORN_IN'), Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='South Africa', type='Location'), type='NATIONALITY'), Relationship(source=Node(id='Elon Reev

In [70]:
graph_documents[0].relationships

[Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='June 28, 1971', type='Date'), type='BORN'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Pretoria', type='Location'), type='BORN_IN'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='South Africa', type='Location'), type='NATIONALITY'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='British', type='Nationality'), type='ANCESTRY'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Pennsylvania Dutch', type='Ethnicity'), type='ANCESTRY'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Maye (Née Haldeman)', type='Person'), type='MOTHER'),
 Relationship(source=Node(id='Maye (Née Haldeman)', type='Person'), target=Node(id='Saskatchewan', type='Location'), type='BORN_IN'),
 Relationship(source=Node(id='Maye (Née Haldeman)', type='Person'), target=Node(i

In [72]:
### Load the dataset of movie

movie_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [74]:
graph.query(movie_query)

[]

In [76]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Person {born: INTEGER, name: STRING}
Movie {title: STRING, released: INTEGER, name: STRING, id: STRING, imdbRating: FLOAT}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Movie)-[:IN_GENRE]->(:Genre)


In [81]:
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(llm, graph=graph, verbose=True)

In [82]:
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7ebb0517f3d0>, cypher_generation_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'schema'], template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7ebb0517fb50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7ebb05e1b0d0>, model_name='Gemma2-9b-it', groq_api_key=SecretStr('**********'))), qa_chain=

In [106]:
response = chain.invoke({"query":"who were the actors and directors of the movie GoldenEye"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "GoldenEye"})<-[:ACTED_IN|DIRECTED]-(p:Person)
RETURN p.name AS personName

Full Context:
[{'personName': 'Pierce Brosnan'}, {'personName': 'Famke Janssen'}, {'personName': 'Sean Bean'}, {'personName': 'Izabella Scorupco'}, {'personName': 'Martin Campbell'}]

> Finished chain.


In [103]:
response["result"]

'Pierce Brosnan, Famke Janssen, Sean Bean, Izabella Scorupco and Martin Campbell  \n'